In [76]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [77]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From Elo_AllMatches",
    con=devengine,)


In [78]:
elo_data['Elo_Diff']=elo_data['Elo_Fav_Elo']-elo_data['Elo_Dog_Elo']
elo_data=elo_data[((elo_data['Elo_Diff']).lt(100))!=True]
elo_data['Winner']=elo_data.apply(lambda x: 'Elo_Fav' if x['Elo_Fav']==x['Winner'] else 'Elo_Dog',axis=1)
#elo_data['Winner_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Odds'],axis=1)
#elo_data['Winner_Est_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Est_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Est_Odds'],axis=1)
#elo_data['Winner_Elo']=elo_data.apply(lambda x: x['Elo_Fav_Elo'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Elo'],axis=1)
elo_data['Fav_Odds']=elo_data[['Elo_Fav_Odds','Elo_Dog_Odds']].min(axis=1)
#elo_data=elo_data[elo_data['Elo_Fav_Odds']<1.8]
train_data=elo_data[elo_data['Date']<'2022-09-01']
test_data=elo_data[elo_data['Date']>='2022-09-01']
test_data_resulted=test_data.reset_index()
test_data=test_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Resulted','Time'])
train_data=train_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Resulted','Time'])


In [79]:
print(len(test_data))
print(len(test_data_resulted))

941
941


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [81]:
X = train_data.drop(["Winner"], axis=1)
y = train_data["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model2 = LogisticRegression(max_iter=20000000)
#model2 = SVC()
model2.fit(X_train, y_train)
train_score2 = model2.score(X_train, y_train)
test_score2 = model2.score(X_test, y_test)
print(train_score2,test_score2)

0.7042518205308903 0.709585121602289


In [82]:
model2.intercept_

array([0.0014674])

In [83]:
pred2=model2.predict(test_data.drop(columns='Winner',axis=1))

In [84]:
len(test_data_resulted)

941

In [85]:
cols = [
    "Date",
    "Prediction",
    "Winner",
    "Elo_Fav",
    "Elo_Fav_Odds",
    "Elo_Dog",
    "Elo_Dog_Odds","Time","Resulted"
    # "Training_Accuracy",
    # "Testing_Accuracy",
]
df = pd.DataFrame(columns=cols)
List = []

In [86]:
for index, tuples in test_data_resulted.iterrows():
        #if index <> len(test_data_resulted):

            values = [
                test_data_resulted["Date"][index],
                pred2[index],
                test_data_resulted["Winner"][index],
                test_data_resulted["Elo_Fav"][index],
                test_data_resulted["Elo_Fav_Odds"][index],
                test_data_resulted["Elo_Dog"][index],
                test_data_resulted["Elo_Dog_Odds"][index],
                test_data_resulted["Time"][index],
                test_data_resulted["Resulted"][index],
                #  "{:.0%}".format(train_score2),
                #  "{:.0%}".format(test_score2),
            ]
            zipped = zip(cols, values)
            a_dictionary = dict(zipped)
            List.append(a_dictionary)

In [87]:
temp = pd.DataFrame(List)
df = pd.concat([df, temp])

In [88]:
df
df[df['Winner']==df['Prediction']]

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted
0,2022-09-01,Elo_Fav,Elo_Fav,Luca Nardi,1.83,Jesper De Jong,1.89,None,None
2,2022-09-01,Elo_Fav,Elo_Fav,Matteo Berrettini,1.11,Hugo Grenier,6.92,None,None
4,2022-09-01,Elo_Fav,Elo_Fav,Gabriel Decamps,1.35,Kasidit Samrej,3.06,None,None
5,2022-09-01,Elo_Fav,Elo_Fav,Andy Murray,1.18,Emilio Nava,4.89,None,None
6,2022-09-01,Elo_Fav,Elo_Fav,Shelby Rogers,1.21,Viktoria Kuzmova,4.52,None,None
...,...,...,...,...,...,...,...,...,...
933,2022-10-19,Elo_Fav,Elo_Fav,Christopher Eubanks,1.43,Kimmer Coppejans,2.69,12:05,False
935,2022-10-19,Elo_Fav,Elo_Fav,Miomir Kecmanovic,1.25,Flavio Cobolli,3.95,02:40,True
936,2022-10-19,Elo_Fav,Elo_Fav,Nuno Borges,1.92,Albert Ramos Vinolas,1.86,00:10,True
938,2022-10-19,Elo_Fav,Elo_Fav,Tommy Paul,1.01,Leo Borg,21.50,04:50,True


In [89]:
exclude_today=df[df['Date']!='2022-10-19'].copy()

In [90]:
exclude_today['Predicton_Odds']=exclude_today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#exclude_today['Winner_Odds']=exclude_today['Winner_Odds'].astype(float)
exclude_today_odds=exclude_today[((exclude_today['Predicton_Odds']).ge(1.8))&((exclude_today['Predicton_Odds']).le(2.5))].copy()
#exclude_today

In [91]:
len(exclude_today_odds)

39

In [92]:
len(exclude_today_odds[exclude_today_odds['Winner']==exclude_today_odds['Prediction']])/len(exclude_today_odds)

0.5897435897435898

In [93]:
today=df[df['Date']>'2022-10-18'].copy()
today['Predicton_Odds']=today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#today=today[((today['Predicton_Odds']).le(1.8))&((today['Predicton_Odds']).ge(1))]

In [94]:
exclude_today_odds

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Predicton_Odds
0,2022-09-01,Elo_Fav,Elo_Fav,Luca Nardi,1.83,Jesper De Jong,1.89,None,None,1.83
64,2022-09-03,Elo_Fav,Elo_Fav,Luca Nardi,1.85,Jerzy Janowicz,1.90,None,None,1.85
93,2022-09-05,Elo_Fav,Elo_Dog,Jurgen Briand,1.82,Thomaz Bellucci,1.88,None,None,1.82
109,2022-09-06,Elo_Dog,Elo_Dog,Danielle Rose Collins,1.97,Aryna Sabalenka,1.86,None,None,1.86
120,2022-09-07,Elo_Fav,Elo_Fav,Stuart Parker,1.96,Bu Yunchaokete,1.79,None,None,1.96
123,2022-09-07,Elo_Dog,Elo_Fav,Casper Ruud,1.99,Matteo Berrettini,1.85,None,None,1.85
134,2022-09-10,Elo_Fav,Elo_Dog,Maria Timofeeva,1.84,Lina Glushko,1.88,None,None,1.84
160,2022-09-12,Elo_Fav,Elo_Fav,Daniil Glinka,1.88,Roy Smith,1.82,None,None,1.88
161,2022-09-12,Elo_Fav,Elo_Fav,Aidan Mayo,1.85,Sam Riffice,1.84,None,None,1.85
172,2022-09-12,Elo_Fav,Elo_Fav,Eugenie Bouchard,1.85,Joanne Zuger,1.94,None,None,1.85


In [95]:
len(today[today['Winner']==today['Prediction']])/len(today)

0.5333333333333333

In [104]:
today[today['Resulted']!='True'].sort_values(by='Time',ascending=True)

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Predicton_Odds
932,2022-10-19,Elo_Fav,Elo_Dog,Max Purcell,1.62,Marc Polmans,2.18,12:05,False,1.62
933,2022-10-19,Elo_Fav,Elo_Fav,Christopher Eubanks,1.43,Kimmer Coppejans,2.69,12:05,False,1.43
915,2022-10-19,Elo_Fav,Elo_Dog,Caroline Garcia,1.28,Rebecca Marino,3.66,12:30,False,1.28
931,2022-10-19,Elo_Fav,Elo_Dog,Radu Albot,1.29,Joris De Loore,3.38,13:30,False,1.29
918,2022-10-19,Elo_Dog,Elo_Fav,Dan Added,2.00,Mats Rosenkranz,1.75,18:00,False,1.75
934,2022-10-19,Elo_Fav,Elo_Dog,Nicolas Jarry,1.89,Roberto Carballes Baena,1.89,18:00,False,1.89
930,2022-10-19,Elo_Fav,Elo_Fav,Gregoire Barrere,1.42,Raphael Collignon,2.73,18:30,False,1.42
912,2022-10-19,Elo_Fav,Elo_Dog,Viktorija Golubic,1.43,Anastasia Gasanova,2.71,19:00,False,1.43
927,2022-10-19,Elo_Fav,Elo_Dog,Lorenzo Sonego,1.29,Sebastian Baez,3.55,19:30,False,1.29
937,2022-10-19,Elo_Fav,Elo_Dog,Cameron Norrie,1.41,Aslan Karatsev,2.87,21:00,False,1.41


In [103]:
today_resulted=today[(today['Resulted']=='True')&((today['Predicton_Odds']).ge(1.8))&((today['Predicton_Odds']).le(2.5))].sort_values(by='Time',ascending=True)
len(today_resulted[today_resulted['Winner']==today_resulted['Prediction']])/len(today_resulted)


1.0

In [102]:
today_resulted

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Predicton_Odds
936,2022-10-19,Elo_Fav,Elo_Fav,Nuno Borges,1.92,Albert Ramos Vinolas,1.86,00:10,True,1.92
